In [1]:
# Import desired libraries
import torch
import torch.nn as nn
import torch.optim as optim
import wandb
import argparse
import multiprocessing

In [2]:
# Import custom modules
from cnn_class import CNN
from utils import *

In [4]:
def main(args):
    # Wandb details
    user = args.wandb_entity
    project = args.wandb_project
    display_name = "test_run"

    if args.wandb_login:
        wandb.init(entity=user, project=project, name=display_name)

    # Define dataset paths
    train_dir = args.data_directory+'/train'
    test_dir = args.data_directory+'/val'

    # Set hyperparameters
    learning_rate = args.learning_rate
    batch_size = args.batch_size
    max_epochs = args.epochs
    img_size = args.img_size
    activation = get_activation(args.activation)

    device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
    print(f"Using device: {device}")
    num_workers = max(1, multiprocessing.cpu_count() - 2) if torch.cuda.is_available() else 4 if torch.backends.mps.is_available() else 1
    use_augmentation = args.use_augmentation
    # Obtain dataloaders
    train_loader, val_loader, test_loader = dataset_split(
        train_dir, test_dir, batch_size=batch_size, num_workers=num_workers,
        augmentation=use_augmentation, img_size=img_size
    )
    # Define custom CNN
    num_filters = args.num_filters
    kernel_size = args.kernel_size
    num_dense = args.num_dense
    dropout_prob = 0.12
    use_dropout = args.use_dropout
    use_batchnorm = args.use_batchnorm
    padding = args.padding
    model = CNN(
        num_filters=num_filters, num_dense=num_dense, kernel_size=kernel_size, padding=padding,
        activation=activation, use_batchnorm=use_batchnorm, use_dropout=use_dropout,
        dropout_prob=dropout_prob, img_size=img_size
    )
    model = model.to(device)
    # Define loss, optimizer, scheduler
    loss_fn = nn.CrossEntropyLoss()
    optimizer = get_optimizer(args.optimizer, model, learning_rate=learning_rate, weight_decay=args.weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
    # Training output
    print(f"Training with 'Custom CNN'")
    print(f"Training Parameters:")
    print(f"Learning Rate: {learning_rate}, num_filters: {num_filters}, kernel_size: {kernel_size}, num_dense: {num_dense}, batch_size: {batch_size}, dropout: {dropout_prob if use_dropout else None}, batchnorm: {use_batchnorm}, data_augmentation: {use_augmentation}")
    train_loop(train_loader, val_loader, model, loss_fn, optimizer, scheduler=scheduler, device=device, max_epochs=max_epochs, patience_stop=10, wandb_log=args.wandb_login)
    if args.wandb_login:
        wandb.finish()

In [5]:
from argparse import Namespace

args = Namespace(
    wandb_project="DA6401_Assignment_2",
    wandb_entity="ee20d201-indian-institute-of-technology-madras",
    data_directory="../../inaturalist_12K",
    epochs=20,
    batch_size=64,
    img_size=224,
    num_filters=[32, 64, 128, 256, 512],
    kernel_size=[3, 3, 3, 3, 3],
    use_augmentation=True,
    num_dense=[512],
    use_dropout=True,
    use_batchnorm=True,
    dropout_prob=0.12,
    padding=1,
    optimizer="adam",
    learning_rate=0.0006,
    weight_decay=0.0007,
    activation="ReLU",
    wandb_login=False,
)

main(args)

Using device: mps
Training with 'Custom CNN'
Training Parameters:
Learning Rate: 0.0006, num_filters: [32, 64, 128, 256, 512], kernel_size: [3, 3, 3, 3, 3], num_dense: [512], batch_size: 64, dropout: 0.12, batchnorm: True, data_augmentation: True


Training Progress:   0%|          | 0/20 [00:00<?, ?epoch/s]

Epoch [1/20] → Train Loss: 4.0813, Train Acc: 15.74% | Val Loss: 2.1955, Val Acc: 17.96%

Current Learning Rate: 0.000600


Training Progress:   5%|▌         | 1/20 [01:39<31:23, 99.12s/epoch]